In [1]:
from src import clear_subs

clear_subs()

In [2]:
from itertools import product
for item in product([[1]]):
    print(item)

([1],)


In [3]:
from src import Love_numbers_for_options_for_models_for_asymptotic_mu_ratios

Love_numbers_for_options_for_models_for_asymptotic_mu_ratios(
    asymptotic_mu_ratios=[0.2, 0.5],
    elasticity_model_names=["PREM"],
    long_term_anelasticity_model_names=[
        "Burgers_asthenosphere__elastic_lithosphere",
        "Peltier_upper_mantle__elastic_lithosphere",
    ],
    short_term_anelasticity_model_names=["Benjamin_Q_PREM"]
)

([[0.2, 0.2, 0.2, 0.2, 1.0], [0.5, 0.5, 0.5, 0.5, 1.0]],)


TypeError: sequence item 0: expected str instance, float found